# Assignment 2: Implicit Surface Reconstruction
## Edoardo Vassallo - S4965918

In [1]:
import numpy as np
import igl
import meshplot as mp

In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [3]:
# Toggle to choose on which mesh to work
working_on_luigi = True

pi, v = igl.read_triangle_mesh("data/" + ("luigi" if working_on_luigi else "cat") + ".off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 1 if working_on_luigi else 6})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1287999…

# Setting up  the constraints

In [4]:
#
# Auxillary code for distance calculations
#

# Function to find the closest point to as point in a set of points
# Brute force implementation
def find_closest_point(point, points):
    distances = np.linalg.norm(points - point, axis=1)
    return np.argmin(distances)

# Function that retrieves the indices of all points in points 
# that are at distance less than h from point.
# Brute force implementation
def closest_points(point, points, radius):
    distances = np.linalg.norm(points - point, axis=1)
    return np.argwhere(distances < radius).flatten()

# Class that implements a spatial index 
# for a set of points
class SpatialIndex:
    
    def __init__(self, points, cell_size):
        self.points = points
        self.inv_cell_size = 1.0/cell_size
        self.grid = {}
        self._build_grid()

    # Get key of a cell
    def _get_key(self, coord):
        return tuple(coord)

    # Get the coordinates of a point in the grid
    def _grid_coords(self, point):
        return np.floor(point * self.inv_cell_size).astype(int)

    # Build the grid
    def _build_grid(self):
        # For each point, compute its coordinates, key 
        # and add it to the grid
        for idx, pt in enumerate(self.points):
            key = self._get_key(self._grid_coords(pt))
            if key not in self.grid:
                self.grid[key] = []
            self.grid[key].append(idx)
    
    # Find the closest point in the grid to a point
    def find_closest_point(self, point, grid_radius=1):
        # Get the grid coordinates of the point
        p_cell = self._grid_coords(point)
        # initialize
        min_dist2 = np.inf
        closest_idx = None

        # For each cell we search in
        for dx in [-grid_radius, 0, grid_radius]:
            for dy in [-grid_radius, 0, grid_radius]:
                for dz in [-grid_radius, 0, grid_radius]:
                    # Access the cell, if it exist
                    neighbor_key = self._get_key(p_cell + np.array([dx, dy, dz]))
                    if neighbor_key in self.grid:
                        # Get the points inside
                        indices = self.grid[neighbor_key]
                        candidates = self.points[indices]
                        # Compute squared distances and get the nearest
                        dists = np.sum((candidates - point)**2, axis=1)
                        min_local_idx = np.argmin(dists)
                        # Check if it is the nearest so far
                        if dists[min_local_idx] < min_dist2:
                            min_dist2 = dists[min_local_idx]
                            closest_idx = indices[min_local_idx]
        return closest_idx

    def closest_points(self, point, radius):
        # Get the grid coordinates of the point
        p_cell = self._grid_coords(point)
        # Get the grid radius in which to search
        grid_radius = int(np.ceil(radius * self.inv_cell_size))
        # initialize
        radius_sq = radius * radius
        result = []

        # For each cell we search in
        for dx in range(-grid_radius, grid_radius+1):
            for dy in range(-grid_radius, grid_radius+1):
                for dz in range(-grid_radius, grid_radius+1):                   
                    # Access the cell, if it exist
                    neighbor_key = self._get_key(p_cell + np.array([dx, dy, dz]))
                    if neighbor_key in self.grid:
                        # Get the points inside
                        indices = self.grid[neighbor_key]
                        candidates = self.points[indices]
                        # Compute squared distances and get the nearest
                        dists = np.sum((candidates - point)**2, axis=1)
                        # Get points in radius and add them to the result
                        in_radius = np.where(dists <= radius_sq)[0]
                        result.extend([indices[i] for i in in_radius])
        return result

In [5]:
# Add here the code to generate the additional points and constraints

# Function to build constraint equations
def build_constraints(points, normals, epsilon, spindex=None):

    # initialize the arrays
    p = np.zeros((3*len(points), 3))
    f = np.zeros(3*len(points))

    for i, pi in enumerate(points):
        # Add constraint f(pi) = 0
        p[3 * i] = pi
        f[3 * i] = 0.0
        # Add pi+N/pi+2N's constraints
        for j, sign in enumerate([1, -1]):
            epsilon_current = epsilon
            while True:
                # Compute pi+N/pi+2N
                pi_N = pi + (sign * epsilon_current * normals[i])

                # Find the closest point to pi+N/pi+2N
                # If we have a spatial index, use it
                if spindex:
                    closest_index = spindex.find_closest_point(pi_N)
                    # DEBUG
                    closest_index_debug = find_closest_point(pi_N, points)  
                    assert (closest_index_debug == closest_index)
                else:
                    closest_index = find_closest_point(pi_N, points)

                # If the point is the same, break
                if np.array_equal(points[closest_index], pi):
                    break
                # Otherwise, update epsilon and repeat
                epsilon_current /= 2
            # Append new point and constraints
            p[(3 * i) + j + 1] = pi_N
            f[(3 * i) + j + 1] = sign * epsilon
    
    p = np.array(p)
    f = np.array(f)
    
    return p, f

In [6]:
# We'll use the bbox diagonal as reference for some of the parameters
# as such, we define a function to compute it, considering also the alignment
# of the cloud of points

def compute_bbox_diag(points, pca_fix=False):
    # Copy for usage with pca_fix
    points_tmp = points.copy()

    if pca_fix:
        # Center the points
        centroid = np.mean(points_tmp, axis=0)
        centered = points_tmp - centroid

        # Compute eigenvectors
        cov = np.cov(centered.T)
        eigvals, eigvecs = np.linalg.eigh(cov) 

        # Sort eigenvectors
        sort_idx = np.argsort(eigvals)[::-1]
        eigvecs = eigvecs[:, sort_idx]

        # Rotate the points
        points_tmp = centered @ eigvecs

    # Compute the bounding box diagonal
    bbox_min = np.min(points_tmp, axis=0)
    bbox_max = np.max(points_tmp, axis=0)
    diag = np.linalg.norm(bbox_max - bbox_min)
    
    return diag, np.array(points_tmp)

In [7]:
# Compute the bounding box diagonal
bbox_diag, test = compute_bbox_diag(pi, pca_fix=working_on_luigi)

# Compute epsilon
epsilon_factor = 0.01
epsilon = epsilon_factor * bbox_diag

# Spatial Index 
spindex_factor = 0.1
spindex_resolution = spindex_factor * bbox_diag
base_spindex = SpatialIndex(pi, cell_size=spindex_resolution)

# Generate the new pi and constraints
p, f = build_constraints(pi, ni, epsilon, base_spindex)

mp.plot(test, shading={"point_size": 1 if working_on_luigi else 6})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.667772…

In [8]:
# Create color vector
colors = np.zeros((p.shape[0], 3))
colors[0::3] = [0, 0, 1]
colors[1::3] = [1, 0, 0]
colors[2::3] = [0, 1, 0]
# Visualize the new point cloud 
mp.plot(p, c=colors, shading={"point_size": 1 if working_on_luigi else 6})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1301333…

# MLS function

In [9]:
def tet_grid_improved(n, points, pca_fix=False):
    # Copy for usage with pca_fix
    points_tmp = points.copy()

    if pca_fix:
        # Center the points
        centroid = np.mean(points_tmp, axis=0)
        centered = points_tmp - centroid

        # Compute eigenvectors
        cov = np.cov(centered.T)
        eigvals, eigvecs = np.linalg.eigh(cov) 

        # Sort eigenvectors
        sort_idx = np.argsort(eigvals)[::-1]
        eigvecs = eigvecs[:, sort_idx]

        # Rotate the points
        points_tmp = centered @ eigvecs

    # Compute the bounding box
    bbox_min = np.min(points_tmp, axis=0)
    bbox_max = np.max(points_tmp, axis=0)

    # Enlarge slightly the bounding box
    bbox_min -= 0.05 * (bbox_max - bbox_min)
    bbox_max += 0.05 * (bbox_max - bbox_min)

    # Generate the grid
    V, T = tet_grid(n, bbox_min, bbox_max)

    if pca_fix:
        # Return to original space
        V = V @ eigvecs.T + centroid

    return V, T

In [10]:
# Resolution
nx = 40
ny = 40
nz = 40

grid_resolution = (nx, ny, nz)

# Generate grid n x n x n
x, T = tet_grid_improved(grid_resolution, p, working_on_luigi)

In [11]:
def wendland_weight(p, p_i, radius):
    # normalized distance
    r = np.linalg.norm(p_i - p, axis=1)
    q = r / radius
    return (1 - q) ** 4 * (4 * q + 1) * (q < 1)

# Return the polynomial basis functions
# depending on the degree required
def pol_basis(x, degree):
    if degree == 0:
        return np.array([1])
    elif degree == 1:
        return np.array([1, x[0], x[1], x[2]])
    elif degree == 2:
        return np.array([1, x[0], x[1], x[2], x[0]**2, x[1]**2, x[2]**2, 
                         x[0]*x[1], x[1]*x[2], x[2]*x[0]])
    else:
        raise ValueError("Degree must be 0, 1, or 2")

# Evaluate MLS for a given point
def evaluate_MLS(point, points, constraints, radius, pol_degree, spindex=None):
    # Check if the spindex is provided
    if spindex:
        neighbors = spindex.closest_points(point, radius)        
        # DEBUG
        neighbors_brute = closest_points(point, points, radius)
        assert set(neighbors) == set(neighbors_brute)
    else:
        neighbors = closest_points(point, points, radius)
    
    # If the number of constraint points is less than twice 
    # the number of polynomial coefficients, return a large positive value
    if len(neighbors) < ((3/2) * pol_degree**2 + (3/2) * pol_degree + 1):
        return 10e6 
    
    # Linear system components:
    # B is the polynomial basis evaluated at the neighbors
    B = np.array([pol_basis(points[i], pol_degree) for i in neighbors])
    # W is the Wendland weight matrix
    W = np.diag(wendland_weight(point, points[neighbors], radius))

    A = B.T @ W @ B
    b = B.T @ W @ constraints[neighbors]
    
    # Solve the linear system
    coeffs = np.linalg.solve(A, b)
    return pol_basis(point, pol_degree) @ coeffs

def evaluate_MLS_on_grid(x, points, constraints, radius, pol_degree, spindex):
    # Evaluate the MLS function on a grid of points
    fx = np.array([evaluate_MLS(xi, points, constraints, radius, pol_degree, spindex) for xi in x])
    return fx

In [12]:
# Parameters
wendland_factor = 0.1
wendlandRadius = wendland_factor * bbox_diag
polyDegree = 2

# Rebuild the spatial index with the new points
full_spindex = SpatialIndex(p, cell_size=spindex_resolution)

# Evaluate the MLS function on the grid
fx = evaluate_MLS_on_grid(x, p, f, wendlandRadius, polyDegree, full_spindex)

In [13]:
# Assign colors
grid_colors = np.zeros((len(x), 3))
grid_colors[fx >= 0] = [1, 0, 0]  # Red = outside
grid_colors[fx <  0] = [0, 1, 0]  # Green = inside

# Visualize the grid
mp.plot(x, c=grid_colors, shading={"point_size": 1 if working_on_luigi else 6})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.4454481…

---

# Marching to extract surface

In [14]:
# Marcing tet to extract surface
sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)

# Plot noisy surface
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1191837…

In [16]:
# Let's filter out smaller, noisy components

# Count number of faces per component
C = igl.face_components(sf)
component_sizes = np.bincount(C)

# Filter out components with less than min_faces
min_faces = 30000
sf_filtered = sf[component_sizes[C] <= min_faces]

# Keep only used vertices
used_vertices = np.unique(sf_filtered)
# Map from old to new vertex ids
reindex = -np.ones(sv.shape[0], dtype=int)
reindex[used_vertices] = np.arange(len(used_vertices))
# Remaining vertices and faces
sv_filtered = sv[used_vertices]
sf_filtered = reindex[sf_filtered]

# Plot the cleaned surface
mp.plot(sv_filtered, sf_filtered, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1313529…